In [8]:
import numpy as np
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.metrics import confusion_matrix, accuracy_score, mean_squared_error
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, RandomizedSearchCV
from sklearn.multioutput import MultiOutputRegressor

In [9]:
def load_data(file_path):
    A = np.loadtxt(file_path)
    X = A[:, :9]    # Input features
    y = A[:, 9:]    # Output labels
    return X, y

In [10]:
# Load all 3 datasets
X_final, y_final = load_data('data/tictac_final.txt')
y_final = y_final[:, 0]

X_single, y_single = load_data('data/tictac_single.txt')
y_single = y_single[:, 0]

X_multi, y_multi = load_data('data/tictac_multi.txt')

In [12]:
# Create the Tic Tac Toe board
board = [' ' for _ in range(9)]

# Function to print the board
def print_board():
    print('-------------')
    for i in range(3):
        print('|', board[i*3], '|', board[i*3 + 1], '|', board[i*3 + 2], '|')
        print('-------------')

# Function to check if a player has won
def check_win(player):
    # Check rows
    for i in range(0, 9, 3):
        if board[i] == board[i+1] == board[i+2] == player:
            return True
    # Check columns
    for i in range(3):
        if board[i] == board[i+3] == board[i+6] == player:
            return True
    # Check diagonals
    if board[0] == board[4] == board[8] == player:
        return True
    if board[2] == board[4] == board[6] == player:
        return True
    return False

# Function to make a move
def make_move(player, position):
    board[position] = player

# Function to get the available moves
def get_available_moves():
    return [i for i, x in enumerate(board) if x == ' ']

# Model-based AI for the 'O' player
def ai_move(board):
    X = np.array([[board.count('X'), board.count('O'), board.count(' '), 
                  board[0], board[1], board[2], board[3], board[4], board[5], board[6], board[7], board[8]]])
    return model.predict(X)[0]

# Function to play the game
def play_game():
    while True:
        print_board()
        if check_win('X'):
            print('You win!')
            break
        if check_win('O'):
            print('You lose!')
            break
        if ' ' not in board:
            print('It\'s a tie!')
            break
        position = int(input('Enter your move (1-9): ')) - 1
        make_move('X', position)
        available_moves = get_available_moves()
        if available_moves:
            ai_move_index = ai_move(board)
            make_move('O', ai_move_index)

def finetune_clf_parameters(X_train, y_train):
    model = KNeighborsClassifier()

    # Define the parameter grid
    param_grid = {'n_neighbors': np.arange(1, 10),
                  'weights': ['uniform', 'distance'],
                  'metric': ['minkowski', 'euclidean']}

    # RandomizedSearchCV to find the best parameters
    randomized_search = RandomizedSearchCV(model, param_distributions=param_grid, n_iter=10, cv=10, random_state=42)
    randomized_search.fit(X_train, y_train)

    # Get the best value of k
    best_params = randomized_search.best_params_
    print(best_params)
        
    return best_params

# Load the trained model
def load_model():
    # Split into training and testing data
    X_train, X_test, y_train, y_test = train_test_split(X_multi, y_multi, test_size=0.2, shuffle=True, random_state=42)

    # Find the best parameters
    best_params = finetune_clf_parameters(X_train, y_train)

    # Define and train model
    model = KNeighborsClassifier(**best_params)
    model.fit(X_train, y_train)
    model = MultiOutputRegressor(KNeighborsRegressor(n_neighbors=3))
    return model

# Start the game
model = load_model()
play_game()

{'weights': 'distance', 'n_neighbors': 9, 'metric': 'euclidean'}
-------------
|   |   |   |
-------------
|   |   |   |
-------------
|   |   |   |
-------------


NotFittedError: This MultiOutputRegressor instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.